In [ ]:
!pip install ray[tune] mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00


In [23]:
import os
import time
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from ray import tune
from ray.tune.search.optuna import OptunaSearch  # Smart search algorithm
import numpy as np # Import numpy for sqrt

# --- 1. The Training Function (The "Minion" Logic) ---
# This function runs on a separate process (Worker)
def train_model(config):
    # Ensure MLflow experiment is set for each worker process
    mlflow.set_experiment("Diabetes_Distributed_tuning")

    # 1. Load Data (Each worker loads its own copy or reads from shared storage)
    data = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

    # 2. Build Model using hyperparams passed in 'config'
    model = RandomForestRegressor(
        n_estimators=config["n_estimators"],
        max_depth=config["max_depth"],
        min_samples_split=config["min_samples_split"]
    )

    # 3. Train
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    # Fix: Calculate RMSE by taking the square root of MSE
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    # 4. Log to MLflow (The "Registry")
    # We create a new run for every trial
    with mlflow.start_run(nested=True):
        mlflow.log_params(config)
        mlflow.log_metric("rmse", rmse)
        # We tag this model so we can find it later
        mlflow.set_tag("mode", "distributed_trial")

    # 5. Report back to Ray (The "Boss")
    tune.report({"rmse": rmse})

# --- 2. The Orchestrator (The "Boss") ---
def run_distributed_pipeline():
    print("🚀 Initializing Ray Cluster...")
    # Define the search space (The Grid)
    search_space = {
        "n_estimators": tune.randint(50, 500),
        "max_depth": tune.randint(2, 20),
        "min_samples_split": tune.randint(2, 10)
    }

    # The mlflow.set_experiment call has been moved to train_model function
    # mlflow.set_experiment("Diabetes_Distributed_tuning")

    print("⚡ Starting Distributed Tuning (Running parallel trials)...")

    # This executes the training function in parallel across available cores
    analysis = tune.run(
        train_model,
        config=search_space,
        metric="rmse",
        mode="min",            # We want to minimize Error
        num_samples=10,        # Run 10 different experiments
        resources_per_trial={"cpu": 1}, # 1 CPU per worker
        verbose=1
    )

    print("\n✅ Tuning Complete.")
    best_trial = analysis.get_best_trial("rmse", "min", "last")
    print(f"🏆 Best Hyperparameters found: {best_trial.config}")
    print(f"📉 Lowest RMSE: {best_trial.last_result['rmse']:.4f}")

    return best_trial.config
# --- 3. Production Deployment Simulation (FIXED) ---
def register_and_serve(best_config):
    print("\n📦 Promoting Best Model to Production Registry...")

    # Retrain final model on all data
    data = load_diabetes()
    model = RandomForestRegressor(
        n_estimators=best_config["n_estimators"],
        max_depth=best_config["max_depth"],
        min_samples_split=best_config["min_samples_split"]
    )
    model.fit(data.data, data.target)

    # Log the final "Production" model
    # We capture the 'run' object to get the ID safely
    with mlflow.start_run(run_name="PRODUCTION_MODEL") as run:
        mlflow.log_params(best_config)
        mlflow.sklearn.log_model(model, "random_forest_model")

        # --- THE FIX ---
        # Instead of guessing the file path, we construct the official URI
        # Format: runs:/<run_id>/<artifact_path>
        run_id = run.info.run_id
        model_uri = f"runs:/{run_id}/random_forest_model"

        print(f"🔒 Model Versioned at: {model_uri}")

        # SIMULATE SERVING
        print("\n🤖 Mock Inference Server Online...")

        # This instructs MLflow to look up the run internally
        loaded_model = mlflow.sklearn.load_model(model_uri)

        sample_data = data.data[0:1]
        prediction = loaded_model.predict(sample_data)
        print(f"   Input: Patient Data [0.03, 0.05, ...]")
        print(f"   Prediction: Disease Progression = {prediction[0]:.2f}")

if __name__ == "__main__":
    # If best_config is already in memory from the previous cell, you can just run:
    # register_and_serve(best_config)

    # Otherwise, re-run the full pipeline:
    best_config = run_distributed_pipeline()
    register_and_serve(best_config)

🚀 Initializing Ray Cluster...
⚡ Starting Distributed Tuning (Running parallel trials)...
+--------------------------------------------------------------------+
| Configuration for experiment     train_model_2026-01-11_21-41-23   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 10                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_2026-01-11_21-41-23
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2026-01-11_20-08-27_551419_858/artifacts/2026-01-11_21-41-23/train_model_2026-01-11_21-41-23/driver_artifacts`

Trial status: 10 PENDING
Current time: 2026-01-11 21:41:23. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+--------

(train_model pid=59963) 2026/01/11 21:41:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
(train_model pid=59963) 2026/01/11 21:41:52 INFO mlflow.store.db.utils: Updating database tables
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Context impl SQLiteImpl.
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Will assume non-transactional DDL.
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
(train_model pid=59963) 2026/01/11 21:41:52 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table



Trial status: 2 RUNNING | 8 PENDING
Current time: 2026-01-11 21:41:53. Total running time: 30s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------------------------+
| Trial name                status       n_estimators     max_depth     min_samples_split |
+-----------------------------------------------------------------------------------------+
| train_model_46308_00000   RUNNING               125            19                     9 |
| train_model_46308_00001   RUNNING               464            11                     7 |
| train_model_46308_00002   PENDING               355            15                     2 |
| train_model_46308_00003   PENDING                72            12                     2 |
| train_model_46308_00004   PENDING                71            17                     9 |
| train_model_46308_00005   PENDING               259            10                     8 |
| train_model_46308_00006   PEN

(train_model pid=59963) 2026/01/11 21:41:53 INFO alembic.runtime.migration: Running upgrade 400f98739977 -> 6953534de441, add step to inputs table
(train_model pid=59963) 2026/01/11 21:41:53 INFO alembic.runtime.migration: Running upgrade 6953534de441 -> bda7b8c39065, increase_model_version_tag_value_limit
(train_model pid=59963) 2026/01/11 21:41:53 INFO alembic.runtime.migration: Running upgrade bda7b8c39065 -> cbc13b556ace, add V3 trace schema columns
(train_model pid=59963) 2026/01/11 21:41:54 INFO alembic.runtime.migration: Running upgrade cbc13b556ace -> 770bee3ae1dd, add assessments table
(train_model pid=59963) 2026/01/11 21:41:54 INFO alembic.runtime.migration: Running upgrade 770bee3ae1dd -> a1b2c3d4e5f6, add spans table
(train_model pid=59963) 2026/01/11 21:41:54 INFO alembic.runtime.migration: Running upgrade a1b2c3d4e5f6 -> de4033877273, create entity_associations table
(train_model pid=59963) 2026/01/11 21:41:54 INFO alembic.runtime.migration: Running upgrade de4033877273 


Trial train_model_46308_00000 completed after 1 iterations at 2026-01-11 21:41:56. Total running time: 33s
+--------------------------------------------------+
| Trial train_model_46308_00000 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                          6.0861 |
| time_total_s                              6.0861 |
| training_iteration                             1 |
| rmse                                     56.4541 |
+--------------------------------------------------+

Trial train_model_46308_00001 completed after 1 iterations at 2026-01-11 21:42:05. Total running time: 41s
+--------------------------------------------------+
| Trial train_model_46308_00001 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         13.6367 |
| time_total_s                            

...

2026-01-11 21:43:55,356	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_2026-01-11_21-41-23' in 0.0128s.



Trial train_model_46308_00008 completed after 1 iterations at 2026-01-11 21:43:55. Total running time: 2min 31s
+--------------------------------------------------+
| Trial train_model_46308_00008 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         8.16817 |
| time_total_s                             8.16817 |
| training_iteration                             1 |
| rmse                                     55.2142 |
+--------------------------------------------------+

Trial status: 10 TERMINATED
Current time: 2026-01-11 21:43:55. Total running time: 2min 32s
Logical resource usage: 1.0/2 CPUs, 0/0 GPUs
Current best trial: 46308_00005 with rmse=51.55647481495812 and params={'n_estimators': 259, 'max_depth': 10, 'min_samples_split': 8}
+---------------------------------------------------------------------------------------------------------------------------------+
| Tria

2026/01/11 21:43:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🔒 Model Versioned at: runs:/f957370d15ca4caaa99739d795554400/random_forest_model

🤖 Mock Inference Server Online...


   Input: Patient Data [0.03, 0.05, ...]
   Prediction: Disease Progression = 194.39
